In [ ]:
##################
#import libs
##################
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
##################
#read file of all.csv
##################
data=pd.read_csv("../input/all.csv",index_col=None )

#Data cleaning (since the data is from web crawler, it has some faults)

In [ ]:
##################
# we remove the copyright data since it is a bad data
##################
data=data[data.content.str.contains("Copyright")==False]

In [ ]:
##################
# check whether there is any null data 
##################
data[data.content.isnull()==True].index.tolist()

In [ ]:
# This is the fault data checked by my eye in csv file, so remove it  
data=data[data.content.str.contains("from Dana, 1904")==False]

In [ ]:
##################
#import NLP lib
##################
import nltk

In [ ]:
##################
# This function will check the rhyme
# we call it in the funtion doTheyRhyme
##################
def rhyme(inp, level):
    entries = nltk.corpus.cmudict.entries()
    syllables = [(word, syl) for word, syl in entries if word == inp]
    rhymes = []
    for (word, syllable) in syllables:
        rhymes += [word for word, pron in entries if pron[-level:] == syllable[-level:]]
    return set(rhymes)

In [ ]:
##################
# decide the last word of the line to check the rhyme 
# call the function rhyme
##################
def doTheyRhyme ( word1, word2 ):
    # first, we don't want to report 'glue' and 'unglue' as rhyming words
    # those kind of rhymes are LAME
    if word1.find ( word2 ) == len(word1) - len ( word2 ):
        return False
    if word2.find ( word1 ) == len ( word2 ) - len ( word1 ): 
        return False

    return word1 in rhyme ( word2, 1 )

# Rhyme, similes, repetitions, alliterations of the first poem 

In [ ]:
##################
# convert all the content of the poem to lower case
##################
data.content=data.content.str.lower()

In [ ]:
##################
# First poem in our csv file
##################
data.content[0]

make it more readable

In [ ]:
##################
# function for checking the rhyme 
# of AAAA , AABB , ABAB , ABBA
##################
def rhymeforpoem(content):
    lin=(content).replace(","," ").replace("."," ").replace(";"," ").replace(":"," ").replace("!"," ")
    line=lin.splitlines()
    i=0
    Set=[]
    Sentence_set=[]
    result=""
    #print ("---last word of the sentence, see the below for the result ---")
    for li in line:
        #print(li)
        Sentence_set.append(li)
        sp=li.split()
        if ("".join(sp[-1:]) is None or  len("".join(sp[-1:])) == 0 or "".join(sp[-1:])==" "):
            continue
        #print (str(i)+" "+"".join(sp[-1:]))
        Set.append("".join(sp[-1:]) )
        if( len(Set)==4 or len(Set)==8 or len(Set)==12):
            if(  doTheyRhyme(Set[len(Set)-4], Set[len(Set)-3] )==True and  doTheyRhyme(Set[len(Set)-3], Set[len(Set)-2] )==True and doTheyRhyme(Set[len(Set)-2], Set[len(Set)-1] )==True):
                result= ("AAAA");
            elif (  doTheyRhyme(Set[len(Set)-4], Set[len(Set)-3] )==True and  doTheyRhyme(Set[len(Set)-3], Set[len(Set)-2] )==False and doTheyRhyme(Set[len(Set)-2], Set[len(Set)-1] )==True):
                result =("AABB");
            elif (  doTheyRhyme(Set[len(Set)-4], Set[len(Set)-2] )==True and  doTheyRhyme(Set[len(Set)-3], Set[len(Set)-1] )==False and doTheyRhyme(Set[len(Set)-3], Set[len(Set)-2] )==False):
                result =("ABAB");
            elif (  doTheyRhyme(Set[len(Set)-4], Set[len(Set)-1] )==True and  doTheyRhyme(Set[len(Set)-3], Set[len(Set)-1] )==False and doTheyRhyme(Set[len(Set)-3], Set[len(Set)-2] )==True):
                result =("ABBA");
        i=i+1
    if(result==""):    
        result= ("--No Rhyme--");
    #print(result)
    return result
print(rhymeforpoem(data.content[0]))


In [ ]:
##################
# count the similes 
# first we replace the , ; : to space 
# and we split lines 
# and then we check whether like or as are in the line
# which are words of similes 
##################

def similes_and_count (Sentence_set):
    Sentence_set=(Sentence_set).replace(","," ").replace("."," ").replace(";"," ").replace(":"," ").replace("!"," ")
    Sentence_set=Sentence_set.splitlines()
    numb=0
    for sen in Sentence_set:
        sen_break = sen.split();
        if (( "like"  in sen_break) or ("as"  in sen_break)):
            #print (sen) 
            numb +=1
    #print('total number of similes: '+str(numb))
    return numb



In [ ]:
##################
# function for alliterations
# first we replace the , ; : to space 
# and we split lines 
# we make sure that the first character repeat again  by checking the stored character
##################

def alliterations (Sentence_set):
    numberofalliter=0
    Sentence_set=(Sentence_set).replace(","," ").replace("."," ").replace(";"," ").replace(":"," ").replace("!"," ")
    Sentence_set=Sentence_set.splitlines()
    #print ("--sentences for alliterations --")
    for sen in Sentence_set:
        sen_break = sen.split();
        if (sen_break is None or  len(sen_break) == 0 or sen_break==" "):
            continue
        counter=1
        threeorfour=1
        last_character=''
        for ele_in in sen_break:
            if (str(ele_in[0])==str(last_character)):
                threeorfour+=1
                if(threeorfour==3):
                    #print (" ".join(sen_break) )
                    numberofalliter+=1
            if(counter==1):
                last_character=ele_in[0]
            counter+=1
    return numberofalliter          
alliterations(data.content[153])

In [ ]:
##################
# testing the function repetitions 
# which will show below
##################
print ("--sentences for repetitions in I Care Not for These Ladies from docx--")
lin2=(data.content[153]).replace(","," ").replace("."," ").replace(";"," ").replace(":"," ").replace("!"," ")
line2=lin2.splitlines()
for sen in line2:
    line2.remove(sen)
    if(sen in line2 and (sen is  None or  len(sen) == 0 or sen==" ")==False  ):
        print(sen)

In [ ]:
##################
# repetitions for whole line
# we have to remove , . ; to chekc the repetitioin
# 
##################
def repetitions(data):    # data is like data.content[153]
    numbb=0
    lin2=(data).replace(","," ").replace("."," ").replace(";"," ").replace(":"," ").replace("!"," ")
    line2=lin2.splitlines()
    for sen in line2:
        line2.remove(sen)
        if(sen in line2 and (sen is  None or  len(sen) == 0 or sen==" ")==False  ):
            #print(sen)
            numbb+=1
    return numbb
repetitions(data.content[153])

In [ ]:
##################
# show the result of those function 
# metioned above
#
##################
number_list=[
238,
497,
547,
58,
411,
78,
404,
247,
207,
101,
356,
434,
476,
560,
256,
180,
495,
18,
40,
473,
541,
41,
159,
286,
132,
290,
568,
352,
80,
46,
68,
50,
441,
463,
156,
252,
437,
540,
448,
375,
254,
276,
178,
281,
429,
237,
71,
530,
129,
144,
511,
343,
133,
203,
435,
255,
72,
438,
235,
37   
]

frame=pd.DataFrame({  'content number':[i for i  in number_list],
                 'poem name': [ data['poem name'][i] for i  in number_list],
                   'No of similes': [ similes_and_count(data.content[i]) for i  in number_list],
                 'type of rhyme':[rhymeforpoem(data.content[i]) for i  in number_list ],
                   'No of alliterations':[alliterations (data.content[i]) for i  in number_list  ],
                   'No of whole line repetitions': [repetitions(data.content[i]) for i  in number_list]
                    
                 })

frame = frame[['content number',  'poem name','No of similes',
               'type of rhyme','No of alliterations',
            'No of whole line repetitions' 
              ]]
frame

#Convert data into feature 

 1. Turn all the sentence into lower case 
 2. Remove punctuation and Line break
 3. Remove the word of the list in doc
 4.  Find the word map using tfidfvectorizer library



In [ ]:
##################
#  cleaning the data 
# by removing those 
# \n \t \r 
##################
data.content=data.content.str.replace('\n', " ")
data.content=data.content.str.replace("\t", " ")
data.content=data.content.str.replace("\r", " ")
data.content=data.content.str.replace(","," ").replace("."," ")


In [ ]:
##################
# remove list in the document
##################
remove_list=["A",
"An",
"The",
"Aboard",
"About",
"Above",
"Absent",
"Across",
"After",
"Against",
"Along",
"Alongside",
"Amid",
"Among",
"Amongst",
"Anti",
"Around",
"As",
"At",
"Before",
"Behind",
"Below",
"Beneath",
"Beside",
"Besides",
"Between",
"Beyond",
"But",
"By",
"Circa",
"Concerning",
"Considering",
"Despite",
"Down",
"During",
"Except",
"Excepting",
"Excluding",
"Failing",
"Following",
"For",
"From",
"Given",
"In",
"Inside",
"Into",
"Like",
"Minus",
"Near",
"Of",
"Off",
"On",
"Onto",
"Opposite",
"Outside",
"Over",
"Past",
"Per",
"Plus",
"Regarding",
"Round",
"Save",
"Since",
"Than",
"Through",
"To",
"Toward",
"Towards",
"Under",
"Underneath",
"Unlike",
"Until",
"Up",
"Upon",
"Versus",
"Via",
"With",
"Within",
"Without"]

In [ ]:
##################
# replace those words with space
##################
for  value in remove_list:
    data.content=data.content.str.replace(value," ")
data.content

In [ ]:
import re
##################
# regular expression, using stemming: try to replace tail of words like ies to y 
##################

In [ ]:
##################
# stemming technique 
# in NLP
##################
data.content = data.content.str.replace("ing( |$)", " ")
data.content = data.content.str.replace("[^a-zA-Z]", " ")
data.content = data.content.str.replace("ies( |$)", "y ")

In [ ]:
##################
# import tfidf lib from sklearn 
##################
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
##################
# using ngram as one word 
# analyzer as word 
##################
vectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, analyzer= 'word')

In [ ]:
##################
# import train test split lib
##################
from sklearn.model_selection import train_test_split

In [ ]:
# if there is any empty data, drop it
data.dropna(inplace=True)

#Split the data: 80 % for training data and  20 % for testing data

In [ ]:

#Split the data: 80 % for training data and  20 % for testing data
data_content_train,data_content_test, data_train_label,data_test_label =train_test_split(data[["content","author","poem name"]],data.type,test_size = 0.2, random_state = 1)


In [ ]:
type(data.type)

In [ ]:
(data_train_label)

In [ ]:
##################
# correct data of test data 
# will use in the result 
##################
data_test_label_for_age=data.ix[data_test_label.index].age
data_train_label_for_age=data.ix[data_train_label.index].age

In [ ]:
##################
# using fit transform to transform data 
# for fitting vector
# we got our word map (which is the most important features of the poem)
##################
train_ = vectorizer.fit_transform(data_content_train.content.as_matrix())
feature_names =vectorizer.get_feature_names()
feature_names
test_ = vectorizer.transform(data_content_test.content.as_matrix())
print("--word map found by algorithm---")
feature_names

In [ ]:
# check if there is any empty poem name in the file
removelist=data_content_train["poem name"].index[data_content_train["poem name"].isnull()==True].tolist()
removelist

#Since our targets contain :
- age ( Modern, Renaissance ) 
- type (Love, Nature, Mythology & Folklore)
#We use two xgb predictors to predict each poem and show the result

In [ ]:
##################
# testing parameters for performance 
##################
from sklearn import preprocessing
label_au = preprocessing.LabelEncoder()
label_author=label_au.fit_transform(data_content_train.author.as_matrix())
label_authorT=label_au.fit_transform(data_content_test.author.as_matrix())

label_poe_name =TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')  
label_poena=label_poe_name.fit_transform(data_content_train["poem name"].as_matrix())
label_poenaT  =label_poe_name.fit_transform(data_content_test["poem name"].as_matrix())



In [ ]:
#label_author=np.reshape(label_author, (label_author.shape[0], 1))
#label_authorT=np.reshape(label_authorT, (label_authorT.shape[0], 1))
#data_content_train.author[data_content_train.author.isnull()==True].index.tolist()

In [ ]:
#a_age.shape

In [ ]:
from numpy import array

In [ ]:
# We try to catch more feature, but it did not make big difference of result
from sklearn.feature_selection import SelectKBest ,chi2
#y = np.array(data_content_train)
ch2 = SelectKBest(chi2, k=2000)
#X_train=ch2.fit_transform(train_, data_train_label.tolist() )
#X_test = ch2.transform(test_)
X_train=train_;
X_test=test_;

In [ ]:
# It did not make big difference of result if we use dense matrix 
#import scipy.sparse as sp
#if(sp.issparse(X_train)==True):
#   X_train = X_train.todense()
#   X_test = X_test.todense()
    

In [ ]:
##################
# import xgb lib 
# and the parameters are used  as below 
##################
import xgboost as xgb

In [ ]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    "eval_metric": 'logloss',
    'silent': 1
}

In [ ]:
xgb_params_age = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    "eval_metric": 'error',
    'silent': 1
}

In [ ]:
xgb_params_author = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'binary:logistic',
    "eval_metric": 'error',
    'silent': 1
}

In [ ]:
##################
# a are the target for training 
# we have to transfrom to the right form of label
##################
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
a=le.fit_transform(data_train_label.as_matrix())

In [ ]:
##################
# a_age are the target of age for training 
# we have to transfrom to the right form of label
##################
le2 = preprocessing.LabelEncoder()
a_age=le2.fit_transform(data_train_label_for_age.as_matrix())

In [ ]:
# check the correctness of the data 
# to see whether there is any null in it
s=pd.DataFrame({'author':label_author})
s.author.ix[s.author.isnull()==True]


s_T=pd.DataFrame({'author':label_authorT})
s_T.author.ix[s_T.author.isnull()==True]


s_age=pd.DataFrame({'age':a_age})
s_age.age.ix[s_age.age.isnull()==True]
#(s_T)

In [ ]:
#a_age=np.reshape(a_age, (a_age.shape[0], 1))
#a_age.shape

In [ ]:
##################
# Dmatrix using for feeding inputs of xgb predictors
##################
dtrain = xgb.DMatrix(X_train, a )
dtest = xgb.DMatrix(X_test)
dtrain_age = xgb.DMatrix(X_train, a_age )
dtest_age = xgb.DMatrix(X_test)

In [ ]:
dtrain_author = xgb.DMatrix(s, s_age.age )
dtest_author = xgb.DMatrix(s_T)
#dtrain_age = xgb.DMatrix(X_train, a_age )
#dtest_age = xgb.DMatrix(X_test)

In [ ]:
##################
# train and predict 
##################
num_boost_rounds = 422
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
result =model.predict(dtest)

In [ ]:
num_boost_rounds = 422
model_age = xgb.train(dict(xgb_params_age, silent=0), dtrain_age, num_boost_round=num_boost_rounds)
result_age =model_age.predict(dtest_age)

In [ ]:
num_boost_rounds = 422
model_author = xgb.train(dict(xgb_params_author, silent=0), dtrain_author, num_boost_round=num_boost_rounds)
result_author =model_author.predict(dtest_author)

In [ ]:
result_author
# we need to do more to convert it into our label, which will be renaissance and modern

In [ ]:
presult=pd.DataFrame(result)
presult_age=pd.DataFrame(result_age)
presult_author=pd.DataFrame(result_author)

In [ ]:
##################
# the data after training is in  decimal
#  we have to tranform it into int for accuracy compare
# then we can inverse it into label 
##################
presult[(presult.values >= 0.5) & (presult.values < 1.5) ]= 1;
presult[(presult.values >= 1.5) & (presult.values < 2.5) ]=2;
presult[(presult.values >= -0.5) & (presult.values < 0.5) ]=0;

presult_age[(presult_age.values >= -0.5) & (presult_age.values < 0.5) ]=0;
presult_age[(presult_age.values >= 0.5) & (presult_age.values < 1.5) ]= 1;

presult_author[(presult_author.values >= -0.5) & (presult_author.values < 0.5) ]=0;
presult_author[(presult_author.values >= 0.5) & (presult_author.values < 1.5) ]= 1;

In [ ]:
presult=presult.astype(int)
presult_age=presult_age.astype(int)
presult_author=presult_author.astype(int)

In [ ]:
result_back=le.inverse_transform(presult.values)
result_back_age=le2.inverse_transform(presult_age.values)
result_back_author=le2.inverse_transform(presult_author.values)
result_back_author.ravel()

In [ ]:
# after conversion 
result_back_age.ravel()

In [ ]:
# accuracy for target type 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(data_test_label, result_back)
accuracy

In [ ]:
# accuracy for target age
accuracy_age = accuracy_score(data_test_label_for_age, result_back_age)
accuracy_age

In [ ]:
# after conversion 
result_back_age

#Prediction result 

In [ ]:
pd.DataFrame({  'poem name': data_content_test["poem name"],
                'correct_data' : data_test_label_for_age+ " " +data_test_label,
                'predict result' : result_back_age.ravel()+" " +result_back.ravel()
                    })